In [ ]:
#FT1 Installing libraries
!pip install transformers
!pip install torch
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#FT2: Importing Bert libraries

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
file_path = '/content/drive/MyDrive/anyone_final_project/alpha2_dataset_cleaned.csv'

df = pd.read_csv(file_path)
df = df.fillna(pd.NA)
df['type'] = df['type'].fillna('missing')
df['manufacturer'] = df['manufacturer'].fillna('missing')
df['sub_category_1'] = df['sub_category_1'].fillna('missing')
df['sub_category_2'] = df['sub_category_2'].fillna('missing')
df['sub_category_3'] = df['sub_category_3'].fillna('missing')
df['sub_category_4'] = df['sub_category_4'].fillna('missing')

print(df.head())
df.info()

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [ ]:
# stemmer, lemmatizer and stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from typing import Optional

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

def remove_extra_new_lines(text):

    if pd.isnull(text):  # check if text is nan
        return ''  # replace with an empty string

    clean_text = [i for i in str(text).splitlines() if i.strip()]
    clean_text = ' '.join(clean_text)
    return clean_text

def remove_extra_whitespace(text: str) -> str:

    spaceless_text = re.sub(r'\s+', ' ', text)
    return spaceless_text

def remove_special_chars(text: str, remove_digits: Optional[bool] = False) -> str:

    if remove_digits:
        pattern = r'[^a-zA-Z\s]'
    else:
        pattern = r'[^a-zA-Z0-9\s]'

    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def normalize_text(text):

    text = remove_extra_new_lines(text)

    text = remove_extra_whitespace(text)

    text = remove_special_chars(text, remove_digits=False)

    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]

    return ' '.join(lemmas)

In [ ]:
normalization = ['name', 'description']
for column in normalization:
    df[column + '_normalized'] = df[column].apply(normalize_text)

print(df.shape)

(50041, 13)


In [ ]:
print(df.iloc[0])


name                                      Duracell - AAA Batteries (4-Pack)
type                                                               HardGood
price                                                                  5.49
description               Compatible with select electronic devices; AAA...
manufacturer                                                       Duracell
url                                           duracell aaa batteries 4 pack
parent_category                                 Connected Home & Housewares
sub_category_1                                                   Housewares
sub_category_2                                          Household Batteries
sub_category_3                                           Alkaline Batteries
sub_category_4                                                      missing
name_normalized                                        duracell aaa battery
description_normalized    compatible select electronic device aaa size d...
Name: 0, dty

In [ ]:
#df['sub_category_1'].fillna('0', inplace=True)
#df['sub_category_2'].fillna('0', inplace=True)
#df['sub_category_3'].fillna('0', inplace=True)
#df['sub_category_4'].fillna('0', inplace=True)
print(df['parent_category'].unique())

['Connected Home & Housewares' 'other' 'Car Electronics & GPS'
 'In-Store Only' 'Musical Instruments' 'Toys' 'Video Games'
 'Cameras & Camcorders' 'Computers & Tablets' 'Appliances' 'Audio'
 'TV & Home Theater' 'Health' 'Name Brands' 'Cell Phones' 'Movies & Music'
 'Magnolia Home Theater' 'Geek Squad' 'Best Buy Gift Cards'
 'H/VG_X360/Games/B2G1_20130602' 'MP Exclusives' 'Wearable Technology'
 'Custom Parts']


In [ ]:
#FT3: Creation of the Dataset Class for BERT

from torch.utils.data import Dataset
import torch

class AlphaDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
import h5py

print(df.head())
df.shape
X = df.drop(columns=['parent_category'])
y = df['parent_category']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

with h5py.File('label_encoder.h5', 'w') as hf:
    hf.create_dataset('label_encoder', data=label_encoder.classes_)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [ ]:
# FT4: Combining 'name' and 'description' columns into a new 'combined_text' column
X_train['combined_text'] = X_train['name'].fillna('') + " " + X_train['description'].fillna('')
X_test['combined_text'] = X_test['name'].fillna('') + " " + X_test['description'].fillna('')


In [ ]:
df.shape

X_1 = df.drop(columns=['sub_category_1'])
y_1 = df['sub_category_1']
y_1.fillna('missing', inplace=True)

label_encoder_1 = LabelEncoder()
y_1_encoded = label_encoder_1.fit_transform(y_1)

with h5py.File('label_encoder_1.h5', 'w') as hf:
    hf.create_dataset('label_encoder_1', data=label_encoder_1.classes_)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1_encoded, test_size=0.2, random_state=42)

In [ ]:
df.shape
X_2 = df.drop(columns=['sub_category_2'])
y_2 = df['sub_category_2']
y_2.fillna('missing', inplace=True)
label_encoder_2 = LabelEncoder()
y_2_encoded = label_encoder_2.fit_transform(y_2)

with h5py.File('label_encoder_2.h5', 'w') as hf:
    hf.create_dataset('label_encoder_2', data=label_encoder_2.classes_)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2_encoded, test_size=0.2, random_state=42)

In [ ]:
df.shape
X_3 = df.drop(columns=['sub_category_3'])
y_3 = df['sub_category_3']
y_3.fillna('missing', inplace=True)

label_encoder_3 = LabelEncoder()
y_3_encoded = label_encoder_3.fit_transform(y_3)

with h5py.File('label_encoder_3.h5', 'w') as hf:
    hf.create_dataset('label_encoder_3', data=label_encoder_3.classes_)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3_encoded, test_size=0.2, random_state=42)

In [ ]:
df.shape
X_4 = df.drop(columns=['sub_category_4'])
y_4 = df['sub_category_4']
y_4.fillna('missing', inplace=True)

label_encoder_4 = LabelEncoder()
y_4_encoded = label_encoder_4.fit_transform(y_4)

with h5py.File('label_encoder_4.h5', 'w') as hf:
    hf.create_dataset('label_encoder_4', data=label_encoder_4.classes_)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4_encoded, test_size=0.2, random_state=42)

In [ ]:
print(X_4.head())


                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [ ]:
print(X_train.shape)
print( X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train['name_normalized'])

(40032, 13)
(10009, 13)
(40032,)
(10009,)
9233                            star fox preowned nintendo
25631    pioneer networkready ultra hd passthrough av h...
19030                     evolve ultimate edition xbox one
12044    joby pro series ultraplate quickrelease plate ...
18967    aluratek bump w home audio speaker system ipod...
                               ...                        
11284    samsung class diag lead curved smart ultra hd ...
44732    hifonics brutus class mono mosfet subwoofer am...
38158    mobile edge premium laptop backpack apple macb...
860                                 presonus presonus gray
15795      insignia portable bluetooth stereo speaker blue
Name: name_normalized, Length: 40032, dtype: object


In [ ]:
y_train

array([20,  1, 20, ...,  6, 16,  1])

In [ ]:
# FT5: Tokenization of training and test data using the 'combined_text' column
train_encodings = tokenizer(X_train['combined_text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test['combined_text'].tolist(), truncation=True, padding=True)

# Creation of training and test datasets
train_dataset = AlphaDataset(train_encodings, y_train)
test_dataset = AlphaDataset(test_encodings, y_test)


In [ ]:
# FT6: Loading the BERT Model and Configuring Training
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    class_report = classification_report(labels, preds)
    conf_matrix = confusion_matrix(labels, preds)

    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'classification_report': class_report,
        'confusion_matrix': conf_matrix
    }

# Loading the pre-trained BERT model for the classification task
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Use the pre-trained base BERT model
    num_labels=len(np.unique(y_encoded))  # Number of labels for classification
)

# Setting up training parameters
training_args = TrainingArguments(
    output_dir='./results',            # Directory to save training results
    num_train_epochs=5,                # Total number of training epochs
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    warmup_steps=500,                  # Number of warm-up steps to adjust the learning rate
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir='./logs',              # Directory to save logs
    evaluation_strategy="epoch"        # Evaluation strategy (can be "no", "steps", or "epoch")
)

# # Creating a Trainer instance, which will be used to train and evaluate the BERT model
trainer = Trainer(
    model=model,                      # The BERT model to be trained
    args=training_args,               # The training arguments
    train_dataset=train_dataset,      # Training dataset
    eval_dataset=test_dataset,        # Evaluation dataset
    compute_metrics=compute_metrics
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# FT7: Training (Fine-Tuning) the BERT Model

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.308200,0.304190,0.949645
2,0.178900,0.253794,0.959636
3,0.157700,0.220584,0.963932
4,0.080000,0.223278,0.967429
5,0.044400,0.208391,0.969527


TrainOutput(global_step=25020, training_loss=0.20577802450465355, metrics={'train_runtime': 2388.6478, 'train_samples_per_second': 83.796, 'train_steps_per_second': 10.475, 'total_flos': 1.460887089611328e+16, 'train_loss': 0.20577802450465355, 'epoch': 5.0})

In [ ]:
# FT8: Saving the model manually after training
# Especifica la ruta donde deseas guardar el modelo
model_path = "/content/model_finetuned"

# Guarda el modelo
model.save_pretrained(model_path)

# Nota: También puedes guardar el Tokenizador si es necesario
tokenizer.save_pretrained(model_path)

('/content/model_finetuned/tokenizer_config.json',
 '/content/model_finetuned/special_tokens_map.json',
 '/content/model_finetuned/vocab.txt',
 '/content/model_finetuned/added_tokens.json')

In [ ]:
evaluation_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", evaluation_results)

NameError: name 'trainer' is not defined

In [ ]:
#Prediccion 0
text = {
    "name": "Trumpet Multimedia - Trumpets That Work 2015 Calendar - Black",
    "description": "TRUMPET MULTIMEDIA Trumpets That Work 2015 Calendar: 2015 calendar; Trumpets That Work design",
    "price": 23.95,
    "type": "HardGood",
    "manufacturer": "Trumpet Multimedia",
}

text['name'] = normalize_text(text['name'])
text['description'] = normalize_text(text['description'])
text['combined_text'] = text['name'] + " " + text['description']

model_path = '/content/drive/MyDrive/anyone_final_project/finetunning_1'

# Cargar el tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Cargar el modelo
model = BertForSequenceClassification.from_pretrained(model_path)


def predict(text):
    # Preparar los tokens de entrada
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Hacer predicción
    model.eval()  # Asegúrate de que el modelo esté en modo evaluación
    with torch.no_grad():  # Desactivar la generación de gradientes
        outputs = model(**inputs)

    # Obtener la predicción de la última capa
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id


In [ ]:
pred = predict(text['combined_text'])


In [ ]:
print(pred)

19


In [ ]:

with h5py.File('label_encoder.h5', 'r') as hf:
    label_encoder_classes = hf['label_encoder'][:]

def compare_predictions(pred, label_encoder_classes):
    if 0 <= pred < len(label_encoder_classes):
        return label_encoder_classes[pred]
    else:
        return 'unknown'

In [ ]:
comp_pred = compare_predictions(pred, label_encoder_classes)
print(comp_pred)

b'Toys'


In [ ]:
comp_pred_str = comp_pred.decode("utf-8")
print(comp_pred_str)

#Aqui termina prediccion 0

Toys


In [ ]:
# model_1 fine tunning bert
X_train_1['parent_category'] = X_train_1['parent_category'].apply(normalize_text)
X_test_1['parent_category'] = X_test_1['parent_category'].apply(normalize_text)

X_train_1['combined_text'] = X_train_1['name'].fillna('') + " " + X_train_1['description'].fillna('')+ " "+ X_train_1['parent_category']
X_test_1['combined_text'] = X_test_1['name'].fillna('') + " " + X_test_1['description'].fillna('')+ " " + X_test_1['parent_category']

In [ ]:
# FT5: Tokenization of training and test data using the 'combined_text' column
train_encodings_1 = tokenizer(X_train_1['combined_text'].tolist(), truncation=True, padding=True)
test_encodings_1 = tokenizer(X_test_1['combined_text'].tolist(), truncation=True, padding=True)

# Creation of training and test datasets
train_dataset_1 = AlphaDataset(train_encodings_1, y_train_1)
test_dataset_1 = AlphaDataset(test_encodings_1, y_test_1)


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Loading the pre-trained BERT model for the classification task
model_1 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Use the pre-trained base BERT model
    num_labels=len(np.unique(y_1_encoded))  # Number of labels for classification
)

# Setting up training parameters
training_args_1 = TrainingArguments(
    output_dir='./results',            # Directory to save training results
    num_train_epochs=5,                # Total number of training epochs
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    warmup_steps=500,                  # Number of warm-up steps to adjust the learning rate
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir='./logs',              # Directory to save logs
    evaluation_strategy="epoch"        # Evaluation strategy (can be "no", "steps", or "epoch")
)

# # Creating a Trainer instance, which will be used to train and evaluate the BERT model
trainer_1 = Trainer(
    model=model_1,                      # The BERT model to be trained
    args=training_args_1,               # The training arguments
    train_dataset=train_dataset_1,      # Training dataset
    eval_dataset=test_dataset_1,        # Evaluation dataset
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer_1.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.214900,0.268487,0.938256
2,0.180700,0.189889,0.959936
3,0.109300,0.186079,0.966031
4,0.091800,0.180614,0.968029
5,0.035800,0.181010,0.970327


TrainOutput(global_step=25020, training_loss=0.23722762163761232, metrics={'train_runtime': 1974.8191, 'train_samples_per_second': 101.356, 'train_steps_per_second': 12.67, 'total_flos': 1.503265851520128e+16, 'train_loss': 0.23722762163761232, 'epoch': 5.0})

In [ ]:

# Especifica la ruta donde deseas guardar el modelo
model_path = "/content/drive/MyDrive/anyone_final_project/finetunning_2"

# Guarda el modelo
model_1.save_pretrained(model_path)

# Nota: También puedes guardar el Tokenizador si es necesario
tokenizer.save_pretrained(model_path)





('/content/drive/MyDrive/anyone_final_project/finetunning_2/tokenizer_config.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_2/special_tokens_map.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_2/vocab.txt',
 '/content/drive/MyDrive/anyone_final_project/finetunning_2/added_tokens.json')

In [ ]:
#Prediccion 1
text_1 = {
    "name": "Trumpet Multimedia - Trumpets That Work 2015 Calendar - Black",
    "description": "TRUMPET MULTIMEDIA Trumpets That Work 2015 Calendar: 2015 calendar; Trumpets That Work design",
    "price": 23.95,
    "type": "HardGood",
    "manufacturer": "Trumpet Multimedia",
}

text_1['name'] = normalize_text(text_1['name'])
text_1['description'] = normalize_text(text_1['description'])
comp_pred_str = normalize_text(comp_pred_str)
print(comp_pred_str)
text_1['combined_text'] = text_1['name'] + " " + text_1['description']+ " " + comp_pred_str
print(text_1)
model_1_path = '/content/drive/MyDrive/anyone_final_project/finetunning_2'

# Cargar el tokenizer
tokenizer_1 = BertTokenizer.from_pretrained(model_1_path)

# Cargar el modelo
model_1 = BertForSequenceClassification.from_pretrained(model_1_path)


def predict_1(text_1):
    # Preparar los tokens de entrada
    inputs = tokenizer_1(text_1, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Hacer predicción
    model_1.eval()  # Asegúrate de que el modelo esté en modo evaluación
    with torch.no_grad():  # Desactivar la generación de gradientes
        outputs = model_1(**inputs)

    # Obtener la predicción de la última capa
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id


toy
{'name': 'trumpet multimedia trumpet work calendar black', 'description': 'trumpet multimedia trumpet work calendar calendar trumpet work design', 'price': 23.95, 'type': 'HardGood', 'manufacturer': 'Trumpet Multimedia', 'combined_text': 'trumpet multimedia trumpet work calendar black trumpet multimedia trumpet work calendar calendar trumpet work design toy'}


In [ ]:
pred_1 = predict_1(text_1['combined_text'])

In [ ]:
print(pred_1)

42


In [ ]:
with h5py.File('label_encoder_1.h5', 'r') as hf:
    label_encoder_classes_1 = hf['label_encoder_1'][:]

def compare_predictions(pred_1, label_encoder_classes_1):
    if 0 <= pred_1 < len(label_encoder_classes_1):
        return label_encoder_classes_1[pred_1]
    else:
        return 'unknown'

In [ ]:
comp_pred_1 = compare_predictions(pred_1, label_encoder_classes_1)
print(comp_pred_1)

b' Games & Drones'


In [ ]:
comp_pred_str_1 = comp_pred_1.decode("utf-8")
print(comp_pred_str_1)

#Aqui termina prediccion 1

 Games & Drones


In [ ]:
# model_2 fine tunning bert
X_train_2['parent_category'] = X_train_2['parent_category'].apply(normalize_text)
X_test_2['parent_category'] = X_test_2['parent_category'].apply(normalize_text)

X_train_2['sub_category_1'] = X_train_2['sub_category_1'].apply(normalize_text)
X_test_2['sub_category_1'] = X_test_2['sub_category_1'].apply(normalize_text)


X_train_2['combined_text'] = X_train_2['name_normalized'].fillna('') + " " + X_train_2['description_normalized'].fillna('') + " "+ X_train_2['parent_category'] + " "+ X_train_2['sub_category_1']
X_test_2['combined_text'] = X_test_2['name_normalized'].fillna('') + " " + X_test_2['description_normalized'].fillna('') + " " + X_test_2['parent_category'] + " "+ X_test_2['sub_category_1']

In [ ]:
#Tokenization model_2
train_encodings_2 = tokenizer(X_train_2['combined_text'].tolist(), truncation=True, padding=True)
test_encodings_2 = tokenizer(X_test_2['combined_text'].tolist(), truncation=True, padding=True)

# Creation of training and test datasets
train_dataset_2 = AlphaDataset(train_encodings_2, y_train_2)
test_dataset_2 = AlphaDataset(test_encodings_2, y_test_2)


In [ ]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Loading the pre-trained BERT model for the classification task
model_2 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Use the pre-trained base BERT model
    num_labels=len(np.unique(y_2_encoded))  # Number of labels for classification
)

# Setting up training parameters
training_args_2 = TrainingArguments(
    output_dir='./results',            # Directory to save training results
    num_train_epochs=5,                # Total number of training epochs
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    warmup_steps=500,                  # Number of warm-up steps to adjust the learning rate
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir='./logs',              # Directory to save logs
    evaluation_strategy="epoch"        # Evaluation strategy (can be "no", "steps", or "epoch")
)

# # Creating a Trainer instance, which will be used to train and evaluate the BERT model
trainer_2 = Trainer(
    model=model_2,                      # The BERT model to be trained
    args=training_args_2,               # The training arguments
    train_dataset=train_dataset_2,      # Training dataset
    eval_dataset=test_dataset_2,        # Evaluation dataset
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer_2.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.655400,0.650263,0.847337
2,0.387400,0.423119,0.889699
3,0.245500,0.332859,0.920472
4,0.154300,0.322940,0.931861
5,0.086200,0.318869,0.935058


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3500 already exists and is non-empty. Saving will proceed but saved res

TrainOutput(global_step=25020, training_loss=0.5096236059610411, metrics={'train_runtime': 1497.3711, 'train_samples_per_second': 133.674, 'train_steps_per_second': 16.709, 'total_flos': 9183059691693120.0, 'train_loss': 0.5096236059610411, 'epoch': 5.0})

In [ ]:
# Especifica la ruta donde deseas guardar el modelo
model_path = "/content/drive/MyDrive/anyone_final_project/finetunning_3"

# Guarda el modelo
model_2.save_pretrained(model_path)

# Nota: También puedes guardar el Tokenizador si es necesario
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/anyone_final_project/finetunning_3/tokenizer_config.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_3/special_tokens_map.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_3/vocab.txt',
 '/content/drive/MyDrive/anyone_final_project/finetunning_3/added_tokens.json')

In [ ]:
#Prediccion 2
text_2 = {
    "name": "Trumpet Multimedia - Trumpets That Work 2015 Calendar - Black",
    "description": "TRUMPET MULTIMEDIA Trumpets That Work 2015 Calendar: 2015 calendar; Trumpets That Work design",
    "price": 23.95,
    "type": "HardGood",
    "manufacturer": "Trumpet Multimedia",
}

text_2['name'] = normalize_text(text_2['name'])
text_2['description'] = normalize_text(text_2['description'])
comp_pred_str = normalize_text(comp_pred_str)
comp_pred_str_1 = normalize_text(comp_pred_str_1)

text_2['combined_text'] = text_2['name'] + " " + text_2['description']+ " " + comp_pred_str + " " + comp_pred_str_1
print(text_2)
model_2_path = '/content/drive/MyDrive/anyone_final_project/finetunning_3'

# Cargar el tokenizer
tokenizer_2 = BertTokenizer.from_pretrained(model_2_path)

# Cargar el modelo
model_2 = BertForSequenceClassification.from_pretrained(model_2_path)


def predict_2(text_2):
    # Preparar los tokens de entrada
    inputs = tokenizer_2(text_2, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Hacer predicción
    model_2.eval()  # Asegúrate de que el modelo esté en modo evaluación
    with torch.no_grad():  # Desactivar la generación de gradientes
        outputs = model_2(**inputs)

    # Obtener la predicción de la última capa
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

{'name': 'trumpet multimedia trumpet work calendar black', 'description': 'trumpet multimedia trumpet work calendar calendar trumpet work design', 'price': 23.95, 'type': 'HardGood', 'manufacturer': 'Trumpet Multimedia', 'combined_text': 'trumpet multimedia trumpet work calendar black trumpet multimedia trumpet work calendar calendar trumpet work design toy game drone'}


In [ ]:
pred_2 = predict_2(text_2['combined_text'])
print(pred_2)

302


In [ ]:
with h5py.File('label_encoder_2.h5', 'r') as hf:
    label_encoder_classes_2 = hf['label_encoder_2'][:]

def compare_predictions(pred_2, label_encoder_classes_2):
    if 0 <= pred_2 < len(label_encoder_classes_2):
        return label_encoder_classes_2[pred_2]
    else:
        return 'unknown'

In [ ]:
comp_pred_2 = compare_predictions(pred_2, label_encoder_classes_2)
print(comp_pred_2)

b' TV'


In [ ]:
comp_pred_str_2 = comp_pred_2.decode("utf-8")
print(comp_pred_str_2)

#Aqui termina prediccion 2

 TV


In [ ]:
X_train_3['parent_category'] = X_train_3['parent_category'].apply(normalize_text)
X_test_3['parent_category'] = X_test_3['parent_category'].apply(normalize_text)

X_train_3['sub_category_1'] = X_train_3['sub_category_1'].apply(normalize_text)
X_test_3['sub_category_1'] = X_test_3['sub_category_1'].apply(normalize_text)

X_train_3['sub_category_2'] = X_train_3['sub_category_2'].apply(normalize_text)
X_test_3['sub_category_2'] = X_test_3['sub_category_2'].apply(normalize_text)


X_train_3['combined_text'] = X_train_3['name_normalized'].fillna('') + " " + X_train_3['description_normalized'].fillna('') + " "+ X_train_3['parent_category'] + " "+ X_train_3['sub_category_1'] + " "+ X_train_3['sub_category_2']
X_test_3['combined_text'] = X_test_3['name_normalized'].fillna('') + " " + X_test_3['description_normalized'].fillna('') + " " + X_test_3['parent_category'] + " "+ X_test_3['sub_category_1'] + " "+ X_test_3['sub_category_2']

In [ ]:
#Tokenization model_3
train_encodings_3 = tokenizer(X_train_3['combined_text'].tolist(), truncation=True, padding=True)
test_encodings_3 = tokenizer(X_test_3['combined_text'].tolist(), truncation=True, padding=True)

# Creation of training and test datasets
train_dataset_3 = AlphaDataset(train_encodings_3, y_train_3)
test_dataset_3 = AlphaDataset(test_encodings_3, y_test_3)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Loading the pre-trained BERT model for the classification task
model_3 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Use the pre-trained base BERT model
    num_labels=len(np.unique(y_3_encoded))  # Number of labels for classification
)

# Setting up training parameters
training_args_3 = TrainingArguments(
    output_dir='./results',            # Directory to save training results
    num_train_epochs=5,                # Total number of training epochs
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    warmup_steps=500,                  # Number of warm-up steps to adjust the learning rate
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir='./logs',              # Directory to save logs
    evaluation_strategy="epoch"        # Evaluation strategy (can be "no", "steps", or "epoch")
)

# # Creating a Trainer instance, which will be used to train and evaluate the BERT model
trainer_3 = Trainer(
    model=model_3,                      # The BERT model to be trained
    args=training_args_3,               # The training arguments
    train_dataset=train_dataset_3,      # Training dataset
    eval_dataset=test_dataset_3,        # Evaluation dataset
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer_3.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.637700,0.610876,0.839944
2,0.343500,0.372319,0.906085
3,0.225300,0.296952,0.930263
4,0.139800,0.295632,0.941253
5,0.090200,0.294697,0.944350


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3500 already exists and is non-empty. Saving will proceed but saved res

TrainOutput(global_step=25020, training_loss=0.46653971724468263, metrics={'train_runtime': 1578.3959, 'train_samples_per_second': 126.812, 'train_steps_per_second': 15.852, 'total_flos': 9489797167518720.0, 'train_loss': 0.46653971724468263, 'epoch': 5.0})

In [ ]:
# Especifica la ruta donde deseas guardar el modelo
model_path = "/content/drive/MyDrive/anyone_final_project/finetunning_4"

# Guarda el modelo
model_3.save_pretrained(model_path)

# Nota: También puedes guardar el Tokenizador si es necesario
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/anyone_final_project/finetunning_4/tokenizer_config.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_4/special_tokens_map.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_4/vocab.txt',
 '/content/drive/MyDrive/anyone_final_project/finetunning_4/added_tokens.json')

In [ ]:
#Prediccion 3
text_3 = {
    "name": "Trumpet Multimedia - Trumpets That Work 2015 Calendar - Black",
    "description": "TRUMPET MULTIMEDIA Trumpets That Work 2015 Calendar: 2015 calendar; Trumpets That Work design",
    "price": 23.95,
    "type": "HardGood",
    "manufacturer": "Trumpet Multimedia",
}

text_3['name'] = normalize_text(text_3['name'])
text_3['description'] = normalize_text(text_3['description'])
comp_pred_str = normalize_text(comp_pred_str)
comp_pred_str_1 = normalize_text(comp_pred_str_1)
comp_pred_str_2 = normalize_text(comp_pred_str_2)

text_3['combined_text'] = text_3['name'] + " " + text_3['description']+ " " + comp_pred_str + " " + comp_pred_str_1 + " " + comp_pred_str_2
print(text_3)
model_3_path = "/content/drive/MyDrive/anyone_final_project/finetunning_4"

# Cargar el tokenizer
tokenizer_3 = BertTokenizer.from_pretrained(model_3_path)

# Cargar el modelo
model_3 = BertForSequenceClassification.from_pretrained(model_3_path)


def predict_3(text_3):
    # Preparar los tokens de entrada
    inputs = tokenizer_3(text_3, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Hacer predicción
    model_3.eval()  # Asegúrate de que el modelo esté en modo evaluación
    with torch.no_grad():  # Desactivar la generación de gradientes
        outputs = model_3(**inputs)

    # Obtener la predicción de la última capa
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

{'name': 'trumpet multimedia trumpet work calendar black', 'description': 'trumpet multimedia trumpet work calendar calendar trumpet work design', 'price': 23.95, 'type': 'HardGood', 'manufacturer': 'Trumpet Multimedia', 'combined_text': 'trumpet multimedia trumpet work calendar black trumpet multimedia trumpet work calendar calendar trumpet work design toy game drone tv'}


In [ ]:
pred_3 = predict_3(text_3['combined_text'])
print(pred_3)

175


In [ ]:
with h5py.File('label_encoder_3.h5', 'r') as hf:
    label_encoder_classes_3 = hf['label_encoder_3'][:]

def compare_predictions_3(pred_3, label_encoder_classes_3):
    if 0 <= pred_3 < len(label_encoder_classes_3):
        return label_encoder_classes_3[pred_3]
    else:
        return 'unknown'

In [ ]:
comp_pred_3 = compare_predictions_3(pred_3, label_encoder_classes_3)
print(comp_pred_3)

b' Movie & Character Toys'


In [ ]:
comp_pred_str_3 = comp_pred_3.decode("utf-8")
print(comp_pred_str_3)

#Aqui termina prediccion 3

 Movie & Character Toys


In [ ]:
# model_4 fine tunning bert
X_train_4['parent_category'] = X_train_4['parent_category'].apply(normalize_text)
X_test_4['parent_category'] = X_test_4['parent_category'].apply(normalize_text)

X_train_4['sub_category_1'] = X_train_4['sub_category_1'].apply(normalize_text)
X_test_4['sub_category_1'] = X_test_4['sub_category_1'].apply(normalize_text)

X_train_4['sub_category_2'] = X_train_4['sub_category_2'].apply(normalize_text)
X_test_4['sub_category_2'] = X_test_4['sub_category_2'].apply(normalize_text)

X_train_4['sub_category_3'] = X_train_4['sub_category_3'].apply(normalize_text)
X_test_4['sub_category_3'] = X_test_4['sub_category_3'].apply(normalize_text)


X_train_4['combined_text'] = X_train_4['name_normalized'].fillna('') + " " + X_train_4['description_normalized'].fillna('') + " "+ X_train_4['parent_category'] + " "+ X_train_4['sub_category_1'] + " "+ X_train_4['sub_category_2'] + " "+ X_train_4['sub_category_3']
X_test_4['combined_text'] = X_test_4['name_normalized'].fillna('') + " " + X_test_4['description_normalized'].fillna('') + " " + X_test_4['parent_category'] + " "+ X_test_4['sub_category_1'] + " "+ X_test_4['sub_category_2'] + " "+ X_test_4['sub_category_3']

In [ ]:
#Tokenization model_4
train_encodings_4 = tokenizer(X_train_4['combined_text'].tolist(), truncation=True, padding=True)
test_encodings_4 = tokenizer(X_test_4['combined_text'].tolist(), truncation=True, padding=True)

# Creation of training and test datasets
train_dataset_4 = AlphaDataset(train_encodings_4, y_train_4)
test_dataset_4 = AlphaDataset(test_encodings_4, y_test_4)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Loading the pre-trained BERT model for the classification task
model_4 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Use the pre-trained base BERT model
    num_labels=len(np.unique(y_4_encoded))  # Number of labels for classification
)

# Setting up training parameters
training_args_4 = TrainingArguments(
    output_dir='./results',            # Directory to save training results
    num_train_epochs=5,                # Total number of training epochs
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    warmup_steps=500,                  # Number of warm-up steps to adjust the learning rate
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir='./logs',              # Directory to save logs
    evaluation_strategy="epoch"        # Evaluation strategy (can be "no", "steps", or "epoch")
)

# # Creating a Trainer instance, which will be used to train and evaluate the BERT model
trainer_4 = Trainer(
    model=model_4,                      # The BERT model to be trained
    args=training_args_4,               # The training arguments
    train_dataset=train_dataset_4,      # Training dataset
    eval_dataset=test_dataset_4,        # Evaluation dataset
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer_4.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.392500,0.368027,0.908882
2,0.312200,0.263175,0.932261
3,0.226800,0.216670,0.946448
4,0.127900,0.216731,0.956239
5,0.109700,0.191630,0.962134


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3500 already exists and is non-empty. Saving will proceed but saved res

TrainOutput(global_step=25020, training_loss=0.30645013392733916, metrics={'train_runtime': 1580.4546, 'train_samples_per_second': 126.647, 'train_steps_per_second': 15.831, 'total_flos': 9579720222057600.0, 'train_loss': 0.30645013392733916, 'epoch': 5.0})

In [ ]:
# Especifica la ruta donde deseas guardar el modelo
model_path = "/content/drive/MyDrive/anyone_final_project/finetunning_5"

# Guarda el modelo
model_4.save_pretrained(model_path)

# Nota: También puedes guardar el Tokenizador si es necesario
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/anyone_final_project/finetunning_5/tokenizer_config.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_5/special_tokens_map.json',
 '/content/drive/MyDrive/anyone_final_project/finetunning_5/vocab.txt',
 '/content/drive/MyDrive/anyone_final_project/finetunning_5/added_tokens.json')

In [ ]:
#Prediccion 4

text_4 = {
    "name": "Trumpet Multimedia - Trumpets That Work 2015 Calendar - Black",
    "description": "TRUMPET MULTIMEDIA Trumpets That Work 2015 Calendar: 2015 calendar; Trumpets That Work design",
    "price": 23.95,
    "type": "HardGood",
    "manufacturer": "Trumpet Multimedia",
}

text_4['name'] = normalize_text(text_4['name'])
text_4['description'] = normalize_text(text_4['description'])
comp_pred_str = normalize_text(comp_pred_str)
comp_pred_str_1 = normalize_text(comp_pred_str_1)
comp_pred_str_2 = normalize_text(comp_pred_str_2)
comp_pred_str_3 = normalize_text(comp_pred_str_3)

text_4['combined_text'] = text_4['name'] + " " + text_4['description']+ " " + comp_pred_str + " " + comp_pred_str_1 + " " + comp_pred_str_2 + " " + comp_pred_str_3
print(text_4)

model_4_path = "/content/drive/MyDrive/anyone_final_project/finetunning_5"

# Cargar el tokenizer
tokenizer_4 = BertTokenizer.from_pretrained(model_4_path)

# Cargar el modelo
model_4 = BertForSequenceClassification.from_pretrained(model_4_path)


def predict_4(text_4):
    # Preparar los tokens de entrada
    inputs = tokenizer_4(text_4, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Hacer predicción
    model_4.eval()  # Asegúrate de que el modelo esté en modo evaluación
    with torch.no_grad():  # Desactivar la generación de gradientes
        outputs = model_4(**inputs)

    # Obtener la predicción de la última capa
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

{'name': 'trumpet multimedia trumpet work calendar black', 'description': 'trumpet multimedia trumpet work calendar calendar trumpet work design', 'price': 23.95, 'type': 'HardGood', 'manufacturer': 'Trumpet Multimedia', 'combined_text': 'trumpet multimedia trumpet work calendar black trumpet multimedia trumpet work calendar calendar trumpet work design toy game drone tv movie character toy'}


In [ ]:
pred_4 = predict_4(text_4['combined_text'])
print(pred_4)

93


In [ ]:
with h5py.File('label_encoder_4.h5', 'r') as hf:
    label_encoder_classes_4 = hf['label_encoder_4'][:]

def compare_predictions_4(pred_4, label_encoder_classes_4):
    if 0 <= pred_4 < len(label_encoder_classes_4):
        return label_encoder_classes_4[pred_4]
    else:
        return 'unknown'

In [ ]:
comp_pred_4 = compare_predictions_4(pred_4, label_encoder_classes_4)
print(comp_pred_4)

b' More Pop Culture Merchandise'


In [ ]:
comp_pred_str_4 = comp_pred_4.decode("utf-8")
print(comp_pred_str_4)

#Aqui termina prediccion 4

 More Pop Culture Merchandise


One Hot Encoder y Scaler


Stage 1

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pickle

# Define columns
categorical_columns = ['type', 'manufacturer']
numerical_columns = ['price']
text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train[categorical_columns] = X_train[categorical_columns].fillna('missing')
X_test[categorical_columns] = X_test[categorical_columns].fillna('missing')

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_columns])
X_test_encoded = encoder.transform(X_test[categorical_columns])


# Save the encoder
with open('encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)

# Information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded:", X_train_encoded.shape)
print("Data type of X_train_encoded:", type(X_train_encoded))
print("Data type of elements in X_train_encoded:", X_train_encoded.dtype)
print("Shape of X_test_encoded:", X_test_encoded.shape)
print("Data type of X_test_encoded:", type(X_test_encoded))
print("Data type of elements in X_test_encoded:", X_test_encoded.dtype)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_test_scaled = scaler.transform(X_test[numerical_columns])

# Save scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Data type of elements in X_train_scaled:", X_train_scaled.dtype)
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Data type of elements in X_test_scaled:", X_test_scaled.dtype)

Shape of X_train_encoded: (40032, 2195)
Data type of X_train_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded: float64
Shape of X_test_encoded: (10009, 2195)
Data type of X_test_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded: float64
Shape of X_train_scaled: (40032, 1)
Data type of elements in X_train_scaled: float64
Shape of X_test_scaled: (10009, 1)
Data type of elements in X_test_scaled: float64


Stage 2

In [ ]:
# Define columns
categorical_columns_1 = ['type', 'manufacturer', 'parent_category']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_1[categorical_columns_1] = X_train_1[categorical_columns_1].fillna('missing')
X_test_1[categorical_columns_1] = X_test_1[categorical_columns_1].fillna('missing')

# One-hot encode categorical features
encoder_1 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_1 = encoder_1.fit_transform(X_train_1[categorical_columns_1])
X_test_encoded_1 = encoder_1.transform(X_test_1[categorical_columns_1])

with open('encoder_1.pkl', 'wb') as file:
    pickle.dump(encoder_1, file)



# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_1:", X_train_encoded_1.shape)
print("Data type of X_train_encoded_1:", type(X_train_encoded_1))
print("Data type of elements in X_train_encoded_1:", X_train_encoded_1.dtype)
print("Shape of X_test_encoded_1:", X_test_encoded_1.shape)
print("Data type of X_test_encoded_1:", type(X_test_encoded_1))
print("Data type of elements in X_test_encoded_1:", X_test_encoded_1.dtype)

# Scale numerical features
scaler_1 = StandardScaler()
X_train_scaled_1 = scaler_1.fit_transform(X_train_1[numerical_columns])
X_test_scaled_1 = scaler_1.transform(X_test_1[numerical_columns])

with open('scaler_1.pkl', 'wb') as file:
    pickle.dump(scaler_1, file)

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_1:", X_train_scaled_1.shape)
print("Data type of elements in X_train_scaled_1:", X_train_scaled_1.dtype)
print("Shape of X_test_scaled_1:", X_test_scaled_1.shape)
print("Data type of elements in X_test_scaled_1:", X_test_scaled_1.dtype)

Shape of X_train_encoded_1: (40032, 2218)
Data type of X_train_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_1: float64
Shape of X_test_encoded_1: (10009, 2218)
Data type of X_test_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_1: float64
Shape of X_train_scaled_1: (40032, 1)
Data type of elements in X_train_scaled_1: float64
Shape of X_test_scaled_1: (10009, 1)
Data type of elements in X_test_scaled_1: float64


Stage 3

In [ ]:
# Define columns
categorical_columns_2 = ['type', 'manufacturer', 'parent_category', 'sub_category_1']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_2[categorical_columns_2] = X_train_2[categorical_columns_2].fillna('missing')
X_test_2[categorical_columns_2] = X_test_2[categorical_columns_2].fillna('missing')

# One-hot encode categorical features
encoder_2 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_2 = encoder_2.fit_transform(X_train_2[categorical_columns_2])
X_test_encoded_2 = encoder_2.transform(X_test_2[categorical_columns_2])

with open('encoder_2.pkl', 'wb') as file:
    pickle.dump(encoder_2, file)


# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_2:", X_train_encoded_2.shape)
print("Data type of X_train_encoded_2:", type(X_train_encoded_2))
print("Data type of elements in X_train_encoded_2:", X_train_encoded_2.dtype)
print("Shape of X_test_encoded_2:", X_test_encoded_2.shape)
print("Data type of X_test_encoded_2:", type(X_test_encoded_2))
print("Data type of elements in X_test_encoded_2:", X_test_encoded_2.dtype)

# Scale numerical features
scaler_2 = StandardScaler()
X_train_scaled_2 = scaler_2.fit_transform(X_train_2[numerical_columns])
X_test_scaled_2 = scaler_2.transform(X_test_2[numerical_columns])

with open('scaler_2.pkl', 'wb') as file:
    pickle.dump(scaler_2, file)


# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_2:", X_train_scaled_2.shape)
print("Data type of elements in X_train_scaled_2:", X_train_scaled_2.dtype)
print("Shape of X_test_scaled_2:", X_test_scaled_2.shape)
print("Data type of elements in X_test_scaled_2:", X_test_scaled_2.dtype)

Shape of X_train_encoded_2: (40032, 2332)
Data type of X_train_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_2: float64
Shape of X_test_encoded_2: (10009, 2332)
Data type of X_test_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_2: float64
Shape of X_train_scaled_2: (40032, 1)
Data type of elements in X_train_scaled_2: float64
Shape of X_test_scaled_2: (10009, 1)
Data type of elements in X_test_scaled_2: float64


Stage 4

In [ ]:
# Define columns
categorical_columns_3 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_3[categorical_columns_3] = X_train_3[categorical_columns_3].fillna('missing')
X_test_3[categorical_columns_3] = X_test_3[categorical_columns_3].fillna('missing')

# One-hot encode categorical features
encoder_3 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_3 = encoder_3.fit_transform(X_train_3[categorical_columns_3])
X_test_encoded_3 = encoder_3.transform(X_test_3[categorical_columns_3])


with open('encoder_3.pkl', 'wb') as file:
    pickle.dump(encoder_3, file)

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_3:", X_train_encoded_3.shape)
print("Data type of X_train_encoded_3:", type(X_train_encoded_3))
print("Data type of elements in X_train_encoded_3:", X_train_encoded_3.dtype)
print("Shape of X_test_encoded_3:", X_test_encoded_3.shape)
print("Data type of X_test_encoded_3:", type(X_test_encoded_3))
print("Data type of elements in X_test_encoded_3:", X_test_encoded_3.dtype)

# Scale numerical features
scaler_3 = StandardScaler()
X_train_scaled_3 = scaler_3.fit_transform(X_train_3[numerical_columns])
X_test_scaled_3 = scaler_3.transform(X_test_3[numerical_columns])

with open('scaler_3.pkl', 'wb') as file:
    pickle.dump(scaler_3, file)


# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_3:", X_train_scaled_3.shape)
print("Data type of elements in X_train_scaled_3:", X_train_scaled_3.dtype)
print("Shape of X_test_scaled_3:", X_test_scaled_3.shape)
print("Data type of elements in X_test_scaled_3:", X_test_scaled_3.dtype)

Shape of X_train_encoded_3: (40032, 2681)
Data type of X_train_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_3: float64
Shape of X_test_encoded_3: (10009, 2681)
Data type of X_test_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_3: float64
Shape of X_train_scaled_3: (40032, 1)
Data type of elements in X_train_scaled_3: float64
Shape of X_test_scaled_3: (10009, 1)
Data type of elements in X_test_scaled_3: float64


Stage 5

In [ ]:
# Define columns
categorical_columns_4 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2', 'sub_category_3']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_4[categorical_columns_4] = X_train_4[categorical_columns_4].fillna('missing')
X_test_4[categorical_columns_4] = X_test_4[categorical_columns_4].fillna('missing')

# One-hot encode categorical features
encoder_4 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_4 = encoder_4.fit_transform(X_train_4[categorical_columns_4])
X_test_encoded_4 = encoder_4.transform(X_test_4[categorical_columns_4])

with open('encoder_4.pkl', 'wb') as file:
    pickle.dump(encoder_4, file)

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_4:", X_train_encoded_4.shape)
print("Data type of X_train_encoded_4:", type(X_train_encoded_4))
print("Data type of elements in X_train_encoded_4:", X_train_encoded_4.dtype)
print("Shape of X_test_encoded_4:", X_test_encoded_4.shape)
print("Data type of X_test_encoded_4:", type(X_test_encoded_4))
print("Data type of elements in X_test_encoded_4:", X_test_encoded_4.dtype)

# Scale numerical features
scaler_4 = StandardScaler()
X_train_scaled_4 = scaler_4.fit_transform(X_train_4[numerical_columns])
X_test_scaled_4 = scaler_4.transform(X_test_4[numerical_columns])

with open('scaler_4.pkl', 'wb') as file:
    pickle.dump(scaler_4, file)


# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_4:", X_train_scaled_4.shape)
print("Data type of elements in X_train_scaled_4:", X_train_scaled_4.dtype)
print("Shape of X_test_scaled_4:", X_test_scaled_4.shape)
print("Data type of elements in X_test_scaled_4:", X_test_scaled_4.dtype)

Shape of X_train_encoded_4: (40032, 2997)
Data type of X_train_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_4: float64
Shape of X_test_encoded_4: (10009, 2997)
Data type of X_test_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_4: float64
Shape of X_train_scaled_4: (40032, 1)
Data type of elements in X_train_scaled_4: float64
Shape of X_test_scaled_4: (10009, 1)
Data type of elements in X_test_scaled_4: float64


In [ ]:
from scipy.sparse import hstack

# Stage 1
X_train_processed = hstack([X_train_encoded, X_train_scaled]).astype(np.float32).toarray()
X_test_processed = hstack([X_test_encoded, X_test_scaled]).astype(np.float32).toarray()

# Stage 2
X_train_processed_1 = hstack([X_train_encoded_1, X_train_scaled_1]).astype(np.float32).toarray()
X_test_processed_1 = hstack([X_test_encoded_1, X_test_scaled_1]).astype(np.float32).toarray()

# Stage 3
X_train_processed_2 = hstack([X_train_encoded_2, X_train_scaled_2]).astype(np.float32).toarray()
X_test_processed_2 = hstack([X_test_encoded_2, X_test_scaled_2]).astype(np.float32).toarray()

# Stage 4
X_train_processed_3 = hstack([X_train_encoded_3, X_train_scaled_3]).astype(np.float32).toarray()
X_test_processed_3 = hstack([X_test_encoded_3, X_test_scaled_3]).astype(np.float32).toarray()

# Stage 5
X_train_processed_4 = hstack([X_train_encoded_4, X_train_scaled_4]).astype(np.float32).toarray()
X_test_processed_4 = hstack([X_test_encoded_4, X_test_scaled_4]).astype(np.float32).toarray()



In [ ]:
# Dim 1
print("Data type of X_train_processed:", X_train_processed.dtype)
print("Data type of X_test_processed:", X_test_processed.dtype)
print("X_train_processed shape:", X_train_processed.shape)
print("X_test_processed shape:", X_test_processed.shape)

Data type of X_train_processed: float32
Data type of X_test_processed: float32
X_train_processed shape: (40032, 2196)
X_test_processed shape: (10009, 2196)


In [ ]:
# Dim 2
print("Data type of X_train_processed_1:", X_train_processed_1.dtype)
print("Data type of X_test_processed_1:", X_test_processed_1.dtype)
print("X_train_processed_1 shape:", X_train_processed_1.shape)
print("X_test_processed_1 shape:", X_test_processed_1.shape)

Data type of X_train_processed_1: float32
Data type of X_test_processed_1: float32
X_train_processed_1 shape: (40032, 2219)
X_test_processed_1 shape: (10009, 2219)


In [ ]:
# Dim 3
print("Data type of X_train_processed_2:", X_train_processed_2.dtype)
print("Data type of X_test_processed_2:", X_test_processed_2.dtype)
print("X_train_processed_2 shape:", X_train_processed_2.shape)
print("X_test_processed_2 shape:", X_test_processed_2.shape)

Data type of X_train_processed_2: float32
Data type of X_test_processed_2: float32
X_train_processed_2 shape: (40032, 2333)
X_test_processed_2 shape: (10009, 2333)


In [ ]:
# Dim 4
print("Data type of X_train_processed_3:", X_train_processed_3.dtype)
print("Data type of X_test_processed_3:",X_test_processed_3.dtype)
print("X_train_processed_3 shape:", X_train_processed_3.shape)
print("X_test_processed_3 shape:", X_test_processed_3.shape)

Data type of X_train_processed_3: float32
Data type of X_test_processed_3: float32
X_train_processed_3 shape: (40032, 2682)
X_test_processed_3 shape: (10009, 2682)


In [ ]:
# Dim 5
print("Data type of X_train_processed_4:", X_train_processed_4.dtype)
print("Data type of X_test_processed_4:", X_test_processed_4.dtype)
print("X_train_processed_4 shape:", X_train_processed_4.shape)
print("X_test_processed_4 shape:", X_test_processed_4.shape)

Data type of X_train_processed_4: float32
Data type of X_test_processed_4: float32
X_train_processed_4 shape: (40032, 2998)
X_test_processed_4 shape: (10009, 2998)


In [ ]:
import numpy as np

# Define file paths to save the arrays
file_paths = {
    'X_train_processed.npy': X_train_processed,
    'X_test_processed.npy': X_test_processed,
    'X_train_processed_1.npy': X_train_processed_1,
    'X_test_processed_1.npy': X_test_processed_1,
    'X_train_processed_2.npy': X_train_processed_2,
    'X_test_processed_2.npy': X_test_processed_2,
    'X_train_processed_3.npy': X_train_processed_3,
    'X_test_processed_3.npy': X_test_processed_3,
    'X_train_processed_4.npy': X_train_processed_4,
    'X_test_processed_4.npy': X_test_processed_4
}

# Save each array
for file_name, array in file_paths.items():
    np.save(file_name, array)


In [ ]:
X_train_name_embeddings_loaded = np.load('X_train_name_embeddings.npy', allow_pickle= True)
X_train_description_embeddings_loaded = np.load('X_train_description_embeddings.npy',allow_pickle= True)
X_test_name_embeddings_loaded = np.load('X_test_name_embeddings.npy', allow_pickle= True)
X_test_description_embeddings_loaded = np.load('X_test_description_embeddings.npy', allow_pickle= True)

# Load the saved NumPy arrays
X_train_processed_loaded = np.load('X_train_processed.npy', allow_pickle=True)
X_test_processed_loaded = np.load('X_test_processed.npy', allow_pickle=True)
X_train_processed_1_loaded = np.load('X_train_processed_1.npy', allow_pickle=True)
X_test_processed_1_loaded = np.load('X_test_processed_1.npy', allow_pickle=True)
X_train_processed_2_loaded = np.load('X_train_processed_2.npy', allow_pickle=True)
X_test_processed_2_loaded = np.load('X_test_processed_2.npy', allow_pickle=True)
X_train_processed_3_loaded = np.load('X_train_processed_3.npy', allow_pickle=True)
X_test_processed_3_loaded = np.load('X_test_processed_3.npy', allow_pickle=True)
X_train_processed_4_loaded = np.load('X_train_processed_4.npy', allow_pickle=True)
X_test_processed_4_loaded = np.load('X_test_processed_4.npy', allow_pickle=True)



In [ ]:
X_train_name_last_hidden = np.array([x[0][-1] for x in X_train_name_embeddings_loaded])
X_train_description_last_hidden = np.array([x[0][-1] for x in X_train_description_embeddings_loaded])
X_test_name_last_hidden = np.array([x[0][-1] for x in X_test_name_embeddings_loaded])
X_test_description_last_hidden = np.array([x[0][-1] for x in X_test_description_embeddings_loaded])

In [ ]:
print("First element of X_train_name_last_hidden:", X_train_name_last_hidden[0])

First element of X_train_name_last_hidden: [ 6.20537519e-01 -1.57028392e-01 -4.39105332e-01  5.87245941e-01
 -4.92567480e-01 -6.75922573e-01  3.66373003e-01 -7.91122556e-01
  6.63730741e-01  4.76211309e-02  4.97472771e-02 -3.77824754e-01
 -5.29451743e-02 -3.52970883e-03 -7.33473897e-01 -2.74465412e-01
 -8.19092095e-02 -1.52533606e-01  7.26244077e-02 -4.86062840e-02
  3.07591200e-01 -1.32735014e-01  8.47112298e-01  2.39423364e-01
  1.34860486e-01  3.51377934e-01 -4.78440404e-01 -1.80202276e-02
 -2.26639926e-01 -3.52599382e-01 -6.21600211e-01 -2.67526209e-01
 -1.92636084e-02  3.99675339e-01  1.55551210e-01 -8.61436129e-02
  3.39647025e-01 -4.60110493e-02 -5.71235001e-01 -4.14561182e-01
 -3.34018916e-01 -1.61173679e-02 -1.93653673e-01  6.32311583e-01
  5.47741950e-02 -6.00057304e-01  6.22461319e-01  3.42181236e-01
 -2.25496083e-01  4.89310294e-01  2.93806463e-01  1.60992384e-01
 -7.31535181e-02  8.67539346e-02  5.76151237e-02  1.21880889e-01
  2.94119507e-01 -4.34998453e-01  1.54902250e-0

In [ ]:
print("Shape of X_train_name_last_hidden:", X_train_name_last_hidden.shape)
print("Data type of X_train_name_last_hidden:", X_train_name_last_hidden.dtype)

print("Shape of X_train_description_last_hidden:", X_train_description_last_hidden.shape)
print("Data type of X_train_description_last_hidden:", X_train_description_last_hidden.dtype)

print("Shape of X_test_name_last_hidden:", X_test_name_last_hidden.shape)
print("Data type of X_test_name_last_hidden:", X_test_name_last_hidden.dtype)

print("Shape of X_test_description_last_hidden:", X_test_description_last_hidden.shape)
print("Data type of X_test_description_last_hidden:", X_test_description_last_hidden.dtype)

Shape of X_train_name_last_hidden: (40032, 768)
Data type of X_train_name_last_hidden: float32
Shape of X_train_description_last_hidden: (40032, 768)
Data type of X_train_description_last_hidden: float32
Shape of X_test_name_last_hidden: (10009, 768)
Data type of X_test_name_last_hidden: float32
Shape of X_test_description_last_hidden: (10009, 768)
Data type of X_test_description_last_hidden: float32


In [ ]:
X_train_concatenated = np.concatenate((X_train_name_last_hidden, X_train_description_last_hidden), axis=1)
X_test_concatenated = np.concatenate((X_test_name_last_hidden, X_test_description_last_hidden), axis=1)

In [ ]:
print("Shape of X_train_concatenated:", X_train_concatenated.shape)
print("Data type of X_train_concatenated:", X_train_concatenated.dtype)

print("Shape of X_test_concatenated:", X_test_concatenated.shape)
print("Data type of X_test_concatenated:", X_test_concatenated.dtype)

Shape of X_train_concatenated: (40032, 1536)
Data type of X_train_concatenated: float32
Shape of X_test_concatenated: (10009, 1536)
Data type of X_test_concatenated: float32


In [ ]:
# Stage 1
X_train_combined = np.concatenate((X_train_processed_loaded, X_train_concatenated), axis=1)
X_test_combined = np.concatenate((X_test_processed_loaded, X_test_concatenated), axis=1)

# Stage 2
X_train_combined_1 = np.concatenate((X_train_processed_1_loaded, X_train_concatenated), axis=1)
X_test_combined_1 = np.concatenate((X_test_processed_1_loaded, X_test_concatenated), axis=1)

# Stage 3
X_train_combined_2 = np.concatenate((X_train_processed_2_loaded, X_train_concatenated), axis=1)
X_test_combined_2 = np.concatenate((X_test_processed_2_loaded, X_test_concatenated), axis=1)

# Stage 4
X_train_combined_3 = np.concatenate((X_train_processed_3_loaded, X_train_concatenated), axis=1)
X_test_combined_3 = np.concatenate((X_test_processed_3_loaded, X_test_concatenated), axis=1)

# Stage 5
X_train_combined_4 = np.concatenate((X_train_processed_4_loaded, X_train_concatenated), axis=1)
X_test_combined_4 = np.concatenate((X_test_processed_4_loaded, X_test_concatenated), axis=1)


In [ ]:
print("Stage 1 - Train combined shape:", X_train_combined.shape)
print("Stage 1 - Test combined shape:", X_test_combined.shape)
print("Stage 2 - Train combined shape:", X_train_combined_1.shape)
print("Stage 2 - Test combined shape:", X_test_combined_1.shape)
print("Stage 3 - Train combined shape:", X_train_combined_2.shape)
print("Stage 3 - Test combined shape:", X_test_combined_2.shape)
print("Stage 4 - Train combined shape:", X_train_combined_3.shape)
print("Stage 4 - Test combined shape:", X_test_combined_3.shape)
print("Stage 5 - Train combined shape:", X_train_combined_4.shape)
print("Stage 5 - Test combined shape:", X_test_combined_4.shape)

Stage 1 - Train combined shape: (40032, 3732)
Stage 1 - Test combined shape: (10009, 3732)
Stage 2 - Train combined shape: (40032, 3755)
Stage 2 - Test combined shape: (10009, 3755)
Stage 3 - Train combined shape: (40032, 3869)
Stage 3 - Test combined shape: (10009, 3869)
Stage 4 - Train combined shape: (40032, 4218)
Stage 4 - Test combined shape: (10009, 4218)
Stage 5 - Train combined shape: (40032, 4534)
Stage 5 - Test combined shape: (10009, 4534)


In [ ]:
#Define num_clases

num_classes = len(np.unique(y_encoded))
num_classes_1 = len(np.unique(y_1_encoded))
num_classes_2 = len(np.unique(y_2_encoded))
num_classes_3 = len(np.unique(y_3_encoded))
num_classes_4 = len(np.unique(y_4_encoded))


In [ ]:
print(num_classes)
print(num_classes_1)
print(num_classes_2)
print(num_classes_3)
print(num_classes_4)

23
114
349
316
162


In [ ]:
#Stage1
from tensorflow.keras import models, layers


from tensorflow.keras import callbacks
from tensorflow.keras.models import save_model
from sklearn.metrics import classification_report

# Define learning rate function
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16:
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Compile the model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model fit
run_1 = model.fit(X_train_combined, y_train, epochs=60, batch_size=32,
                  validation_data=(X_test_combined, y_test),
                  callbacks=[early_stopping, lr_scheduler_callback])

save_model(model, 'model_1_preberttune.h5')

y_pred_probabilities = model.predict(X_test_combined)
y_pred = np.argmax(y_pred_probabilities, axis=1)

loss, accuracy = model.evaluate(X_test_combined, y_test)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


2024-03-21 22:23:08.276598: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 22:23:08.543005: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 22:23:08.543060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 22:23:08.589042: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 22:23:08.682693: I tensorflow/core/platform/cpu_feature_guar

NameError: name 'X_train_combined' is not defined

In [ ]:
#Stage2
# Define learning rate function
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16:
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Create model
model_2 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_1.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_1, activation='softmax')
])

# Compile model
model_2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_2 = model_2.fit(X_train_combined_1, y_train_1, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_1, y_test_1),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_2, 'model_2_preberttune.h5')

# Evaluate model
loss_2, accuracy_2 = model_2.evaluate(X_test_combined_1, y_test_1)
print("Accuracy for Stage 2 model:", accuracy_2)

# Generate classification report
y_pred_probabilities_2 = model_2.predict(X_test_combined_1)
y_pred_2 = np.argmax(y_pred_probabilities_2, axis=1)
report_2 = classification_report(y_test_1, y_pred_2)
print("Classification Report for Stage 2 model:")
print(report_2)



Epoch 1/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.8211 - accuracy: 0.5912 - val_loss: 0.6835 - val_accuracy: 0.8192 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.8864 - accuracy: 0.7609 - val_loss: 0.4522 - val_accuracy: 0.8706 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.7085 - accuracy: 0.7980 - val_loss: 0.3618 - val_accuracy: 0.8892 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6533 - accuracy: 0.8100 - val_loss: 0.3351 - val_accuracy: 0.8971 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6046 - accuracy: 0.8224 - val_loss: 0.3008 - val_accuracy: 0.9075 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5497 - accuracy: 0.8370 - val_loss: 0.2820 - val_accuracy: 0.9127 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:
# Stage 3

def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16:
        return lr * 0.85
    elif epoch < 19:
        return lr * 0.75
    else:
        return lr * 0.7

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)


model_3 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_2.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_2, activation='softmax')
])

# Compile model
model_3.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_3 = model_3.fit(X_train_combined_2, y_train_2, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_2, y_test_2),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_3, 'model_3_preberttune.h5')

# Evaluate model
loss_3, accuracy_3 = model_3.evaluate(X_test_combined_2, y_test_2)
print("Accuracy for Stage 3 model:", accuracy_3)

# Generate classification report
y_pred_probabilities_3 = model_3.predict(X_test_combined_2)
y_pred_3 = np.argmax(y_pred_probabilities_3, axis=1)
report_3 = classification_report(y_test_2, y_pred_3)
print("Classification Report for Stage 3 model:")
print(report_3)



Epoch 1/60
1251/1251 [==============================] - 5s 4ms/step - loss: 2.8616 - accuracy: 0.4597 - val_loss: 1.4320 - val_accuracy: 0.6822 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.5729 - accuracy: 0.6391 - val_loss: 1.0036 - val_accuracy: 0.7520 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.2517 - accuracy: 0.6884 - val_loss: 0.6970 - val_accuracy: 0.8118 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.0994 - accuracy: 0.7125 - val_loss: 0.6239 - val_accuracy: 0.8291 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.9939 - accuracy: 0.7369 - val_loss: 0.5500 - val_accuracy: 0.8410 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.9307 - accuracy: 0.7497 - val_loss: 0.5146 - val_accuracy: 0.8466 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:
# Stage 4
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16:
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Create model
model_4 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_3.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_3, activation='softmax')
])

# Compile model
model_4.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_4 = model_4.fit(X_train_combined_3, y_train_3, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_3, y_test_3),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_4, 'model_4_preberttune.h5')

# Evaluate model
loss_4, accuracy_4 = model_4.evaluate(X_test_combined_3, y_test_3)
print("Accuracy for Stage 4 model:", accuracy_4)

# Generate classification report
y_pred_probabilities_4 = model_4.predict(X_test_combined_3)
y_pred_4 = np.argmax(y_pred_probabilities_4, axis=1)
report_4 = classification_report(y_test_3, y_pred_4)
print("Classification Report for Stage 4 model:")
print(report_4)


Epoch 1/60
1251/1251 [==============================] - 6s 4ms/step - loss: 2.5191 - accuracy: 0.5539 - val_loss: 1.2217 - val_accuracy: 0.7234 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.3264 - accuracy: 0.7036 - val_loss: 0.7747 - val_accuracy: 0.8044 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 1.0495 - accuracy: 0.7438 - val_loss: 0.5684 - val_accuracy: 0.8464 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.9018 - accuracy: 0.7701 - val_loss: 0.5101 - val_accuracy: 0.8475 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.8145 - accuracy: 0.7869 - val_loss: 0.4323 - val_accuracy: 0.8776 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.7491 - accuracy: 0.8002 - val_loss: 0.4041 - val_accuracy: 0.8803 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:
# Stage 5
print("Running Stage 5 model...")
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16:
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)
# Create model
model_5 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_4.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_4, activation='softmax')
])

# Compile model
model_5.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_5 = model_5.fit(X_train_combined_4, y_train_4, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_4, y_test_4),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_5, 'model_5_preberttune.h5')

# Evaluate model
loss_5, accuracy_5 = model_5.evaluate(X_test_combined_4, y_test_4)
print("Accuracy for Stage 5 model:", accuracy_5)

# Generate classification report
y_pred_probabilities_5 = model_5.predict(X_test_combined_4)
y_pred_5 = np.argmax(y_pred_probabilities_5, axis=1)
report_5 = classification_report(y_test_4, y_pred_5)
print("Classification Report for Stage 5 model:")
print(report_5)


Running Stage 5 model...
Epoch 1/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.5423 - accuracy: 0.7581 - val_loss: 0.5663 - val_accuracy: 0.8785 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6838 - accuracy: 0.8587 - val_loss: 0.4186 - val_accuracy: 0.9025 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5449 - accuracy: 0.8736 - val_loss: 0.3294 - val_accuracy: 0.9112 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4665 - accuracy: 0.8844 - val_loss: 0.2741 - val_accuracy: 0.9200 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4180 - accuracy: 0.8925 - val_loss: 0.2524 - val_accuracy: 0.9310 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.3787 - accuracy: 0.8972 - val_loss: 0.2349 - val_accuracy: 0.9314 - lr: 6.3241e-04
Epoch 7/6

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f